# Advanced Matching in rustfuzz

Sometimes, string comparison needs to ignore word order, or handle entirely different sets of words gracefully. This is where token sort ratios and set ratios come in.

In [ ]:
from rustfuzz.fuzz import fuzz_ratio, fuzz_token_sort_ratio, fuzz_token_set_ratio

str1 = 'The quick brown fox'
str2 = 'fox brown quick The'

# Standard ratio cares about order
print(f"Ratio: {fuzz_ratio(str1, str2)}")

# Token sort sorts the words first
print(f"Token Sort Ratio: {fuzz_token_sort_ratio(str1, str2)}")


## Token Set Ratio

If one string is a subset of the other (with extra words), `fuzz_token_set_ratio` calculates the ratio based primarily on the intersection of the words.

In [ ]:
str3 = 'apple banana orange'
str4 = 'apple orange banana grape kiwi'

print(f"Token Set Ratio: {fuzz_token_set_ratio(str3, str4)}")
